## Working Title: Assault and Alcohol Availability in Washington, D.C.

Contributors: Seoho Hahm, Allison Lee

### Table of Contents
1. <a href='#prob'>Problem Statement</a>
2. <a href='#sources'>Data Sources</a>
3. <a href='#collection'>Data Collection</a>
4. <a href='#eda'>Exploratory Data Analysis</a>
5. <a href='#reg'>Regression</a>
6. <a href='#analysis'>Findings and Analysis</a>
7. <a href='#concl'>Conclusions and Recommendations</a>

<a id='prob'></a>
### Problem Statement

ADW: Assault with Deadly Weapon